# 生成ネットワーク

リカレントニューラルネットワーク (RNN) とそのゲート付きセルのバリエーション（例えば、長短期記憶セル (LSTM) やゲート付きリカレントユニット (GRU)）は、言語モデル化の仕組みを提供します。つまり、これらは単語の順序を学習し、シーケンス内の次の単語を予測することができます。この特性により、RNNを使用して、通常のテキスト生成、機械翻訳、さらには画像キャプション生成といった**生成タスク**を実行することが可能になります。

前のユニットで議論したRNNアーキテクチャでは、各RNNユニットが次の隠れ状態を出力として生成していました。しかし、各リカレントユニットにもう一つの出力を追加することで、**シーケンス**（元のシーケンスと同じ長さのもの）を出力することが可能になります。さらに、各ステップで入力を受け取らず、初期状態ベクトルだけを受け取り、それに基づいて出力シーケンスを生成するRNNユニットを使用することもできます。

このノートブックでは、テキスト生成を助けるシンプルな生成モデルに焦点を当てます。簡単のために、**文字レベルのネットワーク**を構築し、文字ごとにテキストを生成します。トレーニング中には、テキストコーパスを取得し、それを文字列のシーケンスに分割する必要があります。


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## キャラクター語彙の構築

キャラクターレベルの生成ネットワークを構築するには、テキストを単語ではなく個々の文字に分割する必要があります。これを実現するためには、異なるトークナイザーを定義する必要があります。


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


データセットからテキストをどのようにエンコードできるかの例を見てみましょう。


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## ジェネレーティブRNNのトレーニング

RNNを使ってテキストを生成する方法は以下の通りです。各ステップで、`nchars`の長さの文字列を入力として取り、ネットワークに対して各入力文字に対する次の出力文字を生成するように求めます。

![単語 'HELLO' を生成するRNNの例を示す画像。](../../../../../translated_images/ja/rnn-generate.56c54afb52f9781d.webp)

実際のシナリオによっては、*end-of-sequence* `<eos>` のような特別な文字を含めることもあります。しかし、今回の場合は無限にテキストを生成するネットワークをトレーニングしたいので、各シーケンスのサイズを`nchars`トークンに固定します。その結果、各トレーニング例は`nchars`の入力と`nchars`の出力（入力シーケンスを1文字左にシフトしたもの）で構成されます。ミニバッチはこのようなシーケンスをいくつかまとめたものになります。

ミニバッチを生成する方法としては、長さ`l`のニューステキストを取り、それから可能なすべての入力-出力の組み合わせを生成します（その組み合わせは`l-nchars`個になります）。これらは1つのミニバッチを形成し、トレーニングステップごとにミニバッチのサイズは異なることになります。


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

ジェネレーター・ネットワークを定義してみましょう。これは、前のユニットで説明した任意のリカレントセル（シンプル、LSTM、またはGRU）を基に構築できます。この例ではLSTMを使用します。

ネットワークは文字を入力として受け取り、語彙サイズが比較的小さいため、埋め込み層は必要ありません。一つのホットエンコードされた入力を直接LSTMセルに渡すことができます。ただし、文字の番号を入力として渡すため、LSTMに渡す前にそれらを一つのホットエンコードに変換する必要があります。これは、`forward`パス中に`one_hot`関数を呼び出すことで行われます。出力エンコーダーは、隠れ状態を一つのホットエンコードされた出力に変換する線形層になります。


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

トレーニング中に生成されたテキストをサンプリングできるようにしたいと考えています。そのために、初期文字列 `start` から始まり、長さ `size` の出力文字列を生成する `generate` 関数を定義します。

その動作は以下の通りです。まず、初期文字列全体をネットワークに通し、出力状態 `s` と次に予測される文字 `out` を取得します。`out` はワンホットエンコードされているため、`argmax` を使用して語彙内の文字 `nc` のインデックスを取得し、`itos` を使って実際の文字を特定し、結果として得られる文字列リスト `chars` に追加します。この1文字を生成するプロセスを `size` 回繰り返し、必要な文字数を生成します。


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

さあ、トレーニングを始めましょう！トレーニングループはこれまでの例とほぼ同じですが、精度を表示する代わりに、1000エポックごとに生成されたテキストのサンプルを表示します。

損失の計算方法には特に注意が必要です。損失を計算するには、ワンホットエンコードされた出力 `out` と、期待されるテキスト `text_out`（文字インデックスのリスト）を使用します。幸いなことに、`cross_entropy` 関数は非正規化されたネットワーク出力を最初の引数として受け取り、クラス番号を2番目の引数として受け取ります。これはまさに私たちが持っているものです。この関数はミニバッチサイズに対する自動平均化も行います。

また、トレーニングを `samples_to_train` サンプルで制限し、待ち時間を短縮します。ぜひ実験してみて、より長いトレーニングを試してみてください。場合によっては、複数のエポックでトレーニングすることも可能です（その場合、このコードの周りに別のループを作成する必要があります）。


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

この例はすでにかなり良いテキストを生成していますが、いくつかの方法でさらに改善することができます：

* **より良いミニバッチ生成**  
  トレーニング用のデータを準備する際、1つのサンプルから1つのミニバッチを生成する方法を採用しました。しかし、この方法は理想的ではありません。ミニバッチのサイズがすべて異なり、テキストが`nchars`より小さい場合にはミニバッチを生成できないこともあります。また、小さなミニバッチではGPUを十分に活用できません。より賢明な方法は、すべてのサンプルから1つの大きなテキストチャンクを取得し、すべての入力-出力ペアを生成してシャッフルし、均等なサイズのミニバッチを生成することです。

* **多層LSTM**  
  LSTMセルを2層または3層試してみるのは理にかなっています。前のユニットで述べたように、LSTMの各層はテキストから特定のパターンを抽出します。文字レベルの生成器の場合、低いLSTM層が音節の抽出を担当し、より高い層が単語や単語の組み合わせを担当すると予想されます。これは、LSTMコンストラクタに層数のパラメータを渡すことで簡単に実装できます。

* **GRUユニット**を試してみて、どちらがより良い結果を出すかを確認することもできます。また、**異なる隠れ層のサイズ**を試してみるのも良いでしょう。隠れ層が大きすぎると過学習を引き起こす可能性があります（例えば、ネットワークがテキストをそのまま学習してしまう）。一方で、サイズが小さすぎると良い結果が得られない可能性があります。


## ソフトテキスト生成と温度

以前の`generate`の定義では、生成されるテキストの次の文字として、常に最も高い確率を持つ文字を選んでいました。この結果、以下の例のように、テキストが同じ文字列の繰り返しになりがちでした。
```
today of the second the company and a second the company ...
```

しかし、次の文字の確率分布を見てみると、いくつかの最も高い確率の間に大きな差がない場合があります。例えば、ある文字の確率が0.2で、別の文字が0.19であるといった具合です。たとえば、'*play*'というシーケンスの次の文字を探す場合、次の文字はスペースでも**e**（単語*player*のように）でも同じくらい適切である可能性があります。

このことから、常に確率が最も高い文字を選ぶのが「公平」とは限らないという結論に至ります。2番目に高い確率の文字を選んでも、意味のあるテキストにつながる可能性があるのです。ネットワークの出力による確率分布から文字を**サンプリング**する方が賢明です。

このサンプリングは、いわゆる**多項分布**を実装する`multinomial`関数を使用して行うことができます。この**ソフト**なテキスト生成を実装する関数は以下のように定義されます。


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

私たちは、**温度**と呼ばれるもう1つのパラメーターを導入しました。これは、最高確率にどれだけ厳密に従うべきかを示すために使用されます。温度が1.0の場合、公平な多項分布サンプリングを行い、温度が無限大に近づくと、すべての確率が等しくなり、次の文字をランダムに選択します。以下の例では、温度を上げすぎるとテキストが無意味になることが観察でき、温度が0に近づくと「循環的な」厳密生成されたテキストに似てくることがわかります。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は一切の責任を負いません。
